# Seldon Core Setup

## Prerequisites

You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [A running Kubernetes cluster with kubectl authenticated](https://kubernetes.io/docs/setup/)
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [Helm client](https://helm.sh/)

## Setup Cluster

In [14]:
!kubectl create namespace seldon

namespace/seldon created


In [15]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "gke_dev-george_us-central1-a_standard-cluster-1" modified.


In [16]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


## Install Ingress

### Ambassador

[Ambassador install](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html#install-ambassador)

**Note:** There are reported gRPC issues with ambassador (see https://github.com/SeldonIO/seldon-core/issues/473).

In [17]:
!helm repo add stable https://kubernetes-charts.storage.googleapis.com/

"stable" has been added to your repositories


In [7]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "jetstack" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈ Happy Helming!⎈ 


In [18]:
!helm install ambassador stable/ambassador --set crds.keep=false

manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
NAME: ambassador
LAST DEPLOYED: Fri Dec  6 11:31:58 2019
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
NOTES:
Congratulations! Y

Check all services are running before proceeding.

In [19]:
!kubectl rollout status deployment.apps/ambassador

Waiting for deployment "ambassador" rollout to finish: 0 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 1 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


In a new terminal run:
    
```kubectl port-forward $(kubectl get pods -n default -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n default 8003:8080```

*or*

### Istio

[Istio install](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html#install-istio-ingress-gateway)

**Note:** Remember to add ```--set istio.enabled=true``` flag when installing Seldon Core with Istio Ingress.

## Install Seldon Core

Follow the [Seldon Core Install documentation](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html).

In [20]:
!kubectl create namespace seldon-system

namespace/seldon-system created


If using Ambassador:

In [21]:
!helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set ambassador.enabled=true --set usageMetrics.enabled=true --namespace seldon-system

NAME: seldon-core
LAST DEPLOYED: Fri Dec  6 11:33:00 2019
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 1
TEST SUITE: None


If using Istio:

In [ ]:
!helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set istio.enabled=true --set usageMetrics.enabled=true --namespace seldon-system

Check all services are running before proceeding.

In [22]:
!kubectl rollout status deploy/seldon-controller-manager -n seldon-system

deployment "seldon-controller-manager" successfully rolled out
